## Byte Pair Encoding

A common tokenizer used for many modern LLMs is the Byte-Pair-Tokenizer. This implementation here is for educational purposes. A performant implementation is availble through the `tiktoken` python package written by OpenAI.

This notebook follows the implementation tutorial from Sebastian Raschka - [Byte Pair Encoding (BPE) From Scratch](https://github.com/rasbt/LLMs-from-scratch/blob/aedad7efc3abf42d9ab5211b841fe87e30b5069c/ch02/05_bpe-from-scratch/bpe-from-scratch.ipynb)

### Main idea

The purpose of a tokenizer is to convert text into an integer representation which can be used for training DNNs. The byte-pair-tokenizer in particular converts text into a byte array.

In [2]:
text = "This is some text"
byte_array = bytearray(text, "utf-8")
print(byte_array)

bytearray(b'This is some text')


If we apply `list()` to the byte array, we get a list of bytes converted to integers back where each character is represented as a separate integer. While this is a valid representation, it results in a lot of individual IDs which we need to train on. Just our short sample text would result in 17 individual token IDs.

In [6]:
ids = list(byte_array)
print(ids)
print("Number of characters:", len(text))
print("Number of token IDs:", len(ids))

[84, 104, 105, 115, 32, 105, 115, 32, 115, 111, 109, 101, 32, 116, 101, 120, 116]
Number of characters: 17
Number of token IDs: 17


Using the byte-pair tokenizer only results in 4 individual token IDs.


In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
bpe_ids = tokenizer.encode(text)
print("Number of token IDs:", len(bpe_ids))

Number of token IDs: 4


Since each byte has 8 bits, it can encode `2**8 = 256` individual values. In BPE, those are usually used to encode single-character tokens. 

We can check this by printing the first 300 tokens using the GPT-2 tokenizer. From index 256 onwards, we find multi-character token encodings.

In [15]:
for i in range(240,270):
    decoded = tokenizer.decode([i])
    print(f"{i}: {decoded}")

240: �
241: �
242: �
243: �
244: �
245: �
246: �
247: �
248: �
249: �
250: �
251: �
252: �
253: �
254: �
255: �
256:  t
257:  a
258: he
259: in
260: re
261: on
262:  the
263: er
264:  s
265: at
266:  w
267:  o
268: en
269:  c


The goal of the tokenizer is to encode the most commonly appearing subwords such as:
```
318: is
617: some
1212: This
2420: text
```

The original algorithm has been described in "[A new algorithm for data compression](http://www.pennelynn.com/Documents/CUJ/HTML/94HTML/19940045.HTM)" by Philip Gage.

### BPE steps

The individual steps to find the encoding are:

1. Identify frequent pairs
    - Iterate over the text and find the most common pair of bytes
2. Replace and Record
    - Replace the found byte pairs with a new ID and add it to a lookup table
    - The size of the final lookup table determines our vocabulary size
3. Repeat those steps until all common pairs have been encoded
    - Iterate over the text using step 1 and 2 until all byte paris which occur more than once have been encoded

We can then use the lookup table to decompress the token representation back to text.

### BPE Encoding Example

We will go through an example using the text `the cat in the hat`.

**Iteration 1**

1. Identify frequent pairs: The byte pair `th` appears twice in the text
2. Replace and Record: Replace `th` with a new token ID that is not yet used and record it in the lookup table, eg ID 256.

    This will result in the text being `<256>e cat in <256>e hat`.

    Our lookup table now has a new entry:
    
    ```
    0: ...
    ...
    256: "th"
    ```

**Iteration 2**

1. Identify frequent pairs: In the partially encoded version of our text the byte pair `<256>e` appears twice.
2. Replace and Record: It gets a new ID 257 and we replace its appearances in the text: `<257> cat in <257> hat`.

    Our updated lookup table now has entries: 
    ```
    256: "th"
    257: "<256>e"
    ```

**Iteration 3**

1. Identify frequent pairs: Next we find that the pair "<257> " appears twice in our text.
2. Replace and Record: We give it a new token ID 258 and replace it in our text: `<258>cat in <258>hat`.

    Our updated lookup table now has entry:
    ```
    256: "th"
    257: "<256>e"
    258: "<257> "
    ```

This procedure continues until any byte pair which appears more than once is encoded.


### Simple BPE Implementation

The python implementation below mimicks the user interface provided by the `tiktoken` library where the encoding procedure described above corresponds to the `train` method.

In [16]:
from collections import Counter, deque
from functools import lru_cache
import json


class BytePairEncoderSimple:
    def __init__(self):
        # Maps token_id to token_str (e.g., {11246: "some"})
        self.vocab = {}
        # Maps token_str to token_id (e.g., {"some": 11246})
        self.inverse_vocab = {}
        # Dictionary of BPE merges: {(token_id1, token_id2): merged_token_id}
        self.bpe_merges = {}

        # For the official OpenAI GPT-2 merges, use a rank dict:
        #  of form {(string_A, string_B): rank}, where lower rank = higher priority
        self.bpe_ranks = {}

    def train(self, text: str, vocab_size: int, allowed_special: set = {"<|endoftext|>"}):
        """
        Train the BPE tokenizer from scratch.

        Args:
            text (str): The training text.
            vocab_size (int): The desired vocabulary size.
            allowed_special (set): A set of special tokens to include.
        """

        # Preprocess: Replace spaces with "Ġ"
        # Note that Ġ is a particularity of the GPT-2 BPE implementation
        # E.g., "Hello world" might be tokenized as ["Hello", "Ġworld"]
        # (GPT-4 BPE would tokenize it as ["Hello", " world"])
        processed_text = []
        for i, char in enumerate(text):
            if char == " " and i != 0:
                processed_text.append("Ġ")
            if char != " ":
                processed_text.append(char)
        processed_text = "".join(processed_text)

        # Initialize vocab with unique characters, including "Ġ" if present
        # Start with the first 256 ASCII characters
        unique_chars = [chr(i) for i in range(256)]
        unique_chars.extend(
            char for char in sorted(set(processed_text)) if char not in unique_chars
        )
        if "Ġ" not in unique_chars:
            unique_chars.append("Ġ")

        self.vocab = {i: char for i, char in enumerate(unique_chars)}
        self.inverse_vocab = {char: i for i, char in self.vocab.items()}

        # Add allowed special tokens
        if allowed_special:
            for token in allowed_special:
                if token not in self.inverse_vocab:
                    new_id = len(self.vocab)
                    self.vocab[new_id] = token
                    self.inverse_vocab[token] = new_id

        # Tokenize the processed_text into token IDs
        token_ids = [self.inverse_vocab[char] for char in processed_text]

        # BPE steps 1-3: Repeatedly find and replace frequent pairs
        for new_id in range(len(self.vocab), vocab_size):
            pair_id = self.find_freq_pair(token_ids, mode="most")
            if pair_id is None:
                break
            token_ids = self.replace_pair(token_ids, pair_id, new_id)
            self.bpe_merges[pair_id] = new_id

        # Build the vocabulary with merged tokens
        for (p0, p1), new_id in self.bpe_merges.items():
            merged_token = self.vocab[p0] + self.vocab[p1]
            self.vocab[new_id] = merged_token
            self.inverse_vocab[merged_token] = new_id

    def load_vocab_and_merges_from_openai(self, vocab_path, bpe_merges_path):
        """
        Load pre-trained vocabulary and BPE merges from OpenAI's GPT-2 files.

        Args:
            vocab_path (str): Path to the vocab file (GPT-2 calls it 'encoder.json').
            bpe_merges_path (str): Path to the bpe_merges file  (GPT-2 calls it 'vocab.bpe').
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            # Convert loaded vocabulary to correct format
            self.vocab = {int(v): k for k, v in loaded_vocab.items()}
            self.inverse_vocab = {k: int(v) for k, v in loaded_vocab.items()}

        # Handle newline character without adding a new token
        if "\n" not in self.inverse_vocab:
            # Use an existing token ID as a placeholder for '\n'
            # Preferentially use "<|endoftext|>" if available
            fallback_token = next(
                (
                    token
                    for token in ["<|endoftext|>", "Ġ", ""]
                    if token in self.inverse_vocab
                ),
                None,
            )
            if fallback_token is not None:
                newline_token_id = self.inverse_vocab[fallback_token]
            else:
                # If no fallback token is available, raise an error
                raise KeyError("No suitable token found in vocabulary to map '\\n'.")

            self.inverse_vocab["\n"] = newline_token_id
            self.vocab[newline_token_id] = "\n"

        # Load GPT-2 merges and store them with an assigned "rank"
        self.bpe_ranks = {}  # reset ranks
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            lines = file.readlines()
            if lines and lines[0].startswith("#"):
                lines = lines[1:]

            rank = 0
            for line in lines:
                pair = tuple(line.strip().split())
                if len(pair) == 2:
                    token1, token2 = pair
                    # If token1 or token2 not in vocab, skip
                    if token1 in self.inverse_vocab and token2 in self.inverse_vocab:
                        self.bpe_ranks[(token1, token2)] = rank
                        rank += 1
                    else:
                        print(
                            f"Skipping pair {pair} as one token is not in the vocabulary."
                        )

    def encode(self, text):
        """
        Encode the input text into a list of token IDs.

        Args:
            text (str): The text to encode.

        Returns:
            List[int]: The list of token IDs.
        """
        tokens = []
        # First split on newlines to preserve them
        lines = text.split("\n")
        for i, line in enumerate(lines):
            if i > 0:
                tokens.append("\n")  # Add newline token separately
            words = line.split()
            for j, word in enumerate(words):
                if j == 0:
                    if i > 0:  # Start of a new line but not the first line
                        tokens.append("Ġ" + word)  # Ensure it's marked as a new segment
                    else:
                        tokens.append(word)
                else:
                    # Prefix words in the middle of a line with "Ġ"
                    tokens.append("Ġ" + word)

        token_ids = []
        for token in tokens:
            if token in self.inverse_vocab:
                # token is contained in the vocabulary as is
                token_ids.append(self.inverse_vocab[token])
            else:
                # Attempt to handle subword tokenization via BPE
                sub_token_ids = self.tokenize_with_bpe(token)
                token_ids.extend(sub_token_ids)

        return token_ids

    def tokenize_with_bpe(self, token):
        """
        Tokenize a single token using BPE merges.

        Args:
            token (str): The token to tokenize.

        Returns:
            List[int]: The list of token IDs after applying BPE.
        """
        # Tokenize the token into individual characters (as initial token IDs)
        token_ids = [self.inverse_vocab.get(char, None) for char in token]
        if None in token_ids:
            missing_chars = [char for char, tid in zip(token, token_ids) if tid is None]
            raise ValueError(f"Characters not found in vocab: {missing_chars}")

        # If we haven't loaded OpenAI's GPT-2 merges, use my approach
        if not self.bpe_ranks:
            can_merge = True
            while can_merge and len(token_ids) > 1:
                can_merge = False
                new_tokens = []
                i = 0
                while i < len(token_ids) - 1:
                    pair = (token_ids[i], token_ids[i + 1])
                    if pair in self.bpe_merges:
                        merged_token_id = self.bpe_merges[pair]
                        new_tokens.append(merged_token_id)
                        # Uncomment for educational purposes:
                        # print(f"Merged pair {pair} -> {merged_token_id} ('{self.vocab[merged_token_id]}')")
                        i += 2  # Skip the next token as it's merged
                        can_merge = True
                    else:
                        new_tokens.append(token_ids[i])
                        i += 1
                if i < len(token_ids):
                    new_tokens.append(token_ids[i])
                token_ids = new_tokens
            return token_ids

        # Otherwise, do GPT-2-style merging with the ranks:
        # 1) Convert token_ids back to string "symbols" for each ID
        symbols = [self.vocab[id_num] for id_num in token_ids]

        # Repeatedly merge all occurrences of the lowest-rank pair
        while True:
            # Collect all adjacent pairs
            pairs = set(zip(symbols, symbols[1:]))
            if not pairs:
                break

            # Find the pair with the best (lowest) rank
            min_rank = float("inf")
            bigram = None
            for p in pairs:
                r = self.bpe_ranks.get(p, float("inf"))
                if r < min_rank:
                    min_rank = r
                    bigram = p

            # If no valid ranked pair is present, we're done
            if bigram is None or bigram not in self.bpe_ranks:
                break

            # Merge all occurrences of that pair
            first, second = bigram
            new_symbols = []
            i = 0
            while i < len(symbols):
                # If we see (first, second) at position i, merge them
                if (
                    i < len(symbols) - 1
                    and symbols[i] == first
                    and symbols[i + 1] == second
                ):
                    new_symbols.append(first + second)  # merged symbol
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            symbols = new_symbols

            if len(symbols) == 1:
                break

        # Finally, convert merged symbols back to IDs
        merged_ids = [self.inverse_vocab[sym] for sym in symbols]
        return merged_ids

    def decode(self, token_ids):
        """
        Decode a list of token IDs back into a string.

        Args:
            token_ids (List[int]): The list of token IDs to decode.

        Returns:
            str: The decoded string.
        """
        decoded_string = ""
        for i, token_id in enumerate(token_ids):
            if token_id not in self.vocab:
                raise ValueError(f"Token ID {token_id} not found in vocab.")
            token = self.vocab[token_id]
            if token == "\n":
                if decoded_string and not decoded_string.endswith(" "):
                    decoded_string += " "  # Add space if not present before a newline
                decoded_string += token
            elif token.startswith("Ġ"):
                decoded_string += " " + token[1:]
            else:
                decoded_string += token
        return decoded_string

    def save_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Save the vocabulary and BPE merges to JSON files.

        Args:
            vocab_path (str): Path to save the vocabulary.
            bpe_merges_path (str): Path to save the BPE merges.
        """
        # Save vocabulary
        with open(vocab_path, "w", encoding="utf-8") as file:
            json.dump(self.vocab, file, ensure_ascii=False, indent=2)

        # Save BPE merges as a list of dictionaries
        with open(bpe_merges_path, "w", encoding="utf-8") as file:
            merges_list = [
                {"pair": list(pair), "new_id": new_id}
                for pair, new_id in self.bpe_merges.items()
            ]
            json.dump(merges_list, file, ensure_ascii=False, indent=2)

    def load_vocab_and_merges(self, vocab_path, bpe_merges_path):
        """
        Load the vocabulary and BPE merges from JSON files.

        Args:
            vocab_path (str): Path to the vocabulary file.
            bpe_merges_path (str): Path to the BPE merges file.
        """
        # Load vocabulary
        with open(vocab_path, "r", encoding="utf-8") as file:
            loaded_vocab = json.load(file)
            self.vocab = {int(k): v for k, v in loaded_vocab.items()}
            self.inverse_vocab = {v: int(k) for k, v in loaded_vocab.items()}

        # Load BPE merges
        with open(bpe_merges_path, "r", encoding="utf-8") as file:
            merges_list = json.load(file)
            for merge in merges_list:
                pair = tuple(merge["pair"])
                new_id = merge["new_id"]
                self.bpe_merges[pair] = new_id

    @lru_cache(maxsize=None)
    def get_special_token_id(self, token):
        return self.inverse_vocab.get(token, None)

    @staticmethod
    def find_freq_pair(token_ids, mode="most"):
        pairs = Counter(zip(token_ids, token_ids[1:]))

        if not pairs:
            return None

        if mode == "most":
            return max(pairs.items(), key=lambda x: x[1])[0]
        elif mode == "least":
            return min(pairs.items(), key=lambda x: x[1])[0]
        else:
            raise ValueError("Invalid mode. Choose 'most' or 'least'.")

    @staticmethod
    def replace_pair(token_ids, pair_id, new_id):
        dq = deque(token_ids)
        replaced = []

        while dq:
            current = dq.popleft()
            if dq and (current, dq[0]) == pair_id:
                replaced.append(new_id)
                # Remove the 2nd token of the pair, 1st was already removed
                dq.popleft()
            else:
                replaced.append(current)

        return replaced

### Testing implementation through example

We can use the text `the-verdict.txt` downloaded in chapter 2 to test our implementation.

In [17]:
with open("the-verdict.txt", "r", encoding="utf-8") as file:
    text = file.read()

We will initialize our BPE Implementation with a vocabulary size of 1000. Given the default dictionary is of size 255 (the individual characters), we can only encode 745 additional byte-pairs.

In [19]:
bpe = BytePairEncoderSimple()
bpe.train(text, vocab_size=1000, allowed_special={"<|endoftext|>"})
print("Size of vocabulary:", len(bpe.vocab))
print("Size of BPE merges:", len(bpe.bpe_merges))

Size of vocabulary: 1000
Size of BPE merges: 742


We can see that our algorithm found additional 742 byte-pairs to encode.

We can now use our trained tokenizer to encode and decode a different text.

In [23]:
input_text = "Jack embraced beauty through art and life."
token_ids = bpe.encode(input_text)
print("Token IDs:", token_ids)
print("Decoded text:", bpe.decode(token_ids))

Token IDs: [424, 256, 654, 531, 302, 311, 256, 296, 97, 465, 121, 595, 841, 116, 287, 466, 256, 326, 972, 46]
Decoded text: Jack embraced beauty through art and life.


We can print the replacements with tokenIDs explicitly to follow the procedure better.

In [26]:
for token_id in token_ids:
    print(f"{token_id} -> {bpe.decode([token_id])}")

424 -> Jack
256 ->  
654 -> em
531 -> br
302 -> ac
311 -> ed
256 ->  
296 -> be
97 -> a
465 -> ut
121 -> y
595 ->  through
841 ->  ar
116 -> t
287 ->  a
466 -> nd
256 ->  
326 -> li
972 -> fe
46 -> .


The implementation also contains methods to save and restore the trained tokenizer which can be done as follows.

In [27]:
# Save trained tokenizer
bpe.save_vocab_and_merges(
    vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt"
)

# Load trained tokenizer
bpe.load_vocab_and_merges(vocab_path="vocab.json", bpe_merges_path="bpe_merges.txt")

### Loading OpenAI's trained tokenizer

Given the implementation is compatible with the OpenAI implementation, we can also load their tokenizer files.

In [28]:
import os
import urllib.request

# Download files if not already present in this directory
def download_file_if_absent(url, filename, search_dirs):
    for directory in search_dirs:
        file_path = os.path.join(directory, filename)
        if os.path.exists(file_path):
            print(f"{filename} already exists in {file_path}")
            return file_path

    target_path = os.path.join(search_dirs[0], filename)
    try:
        with (
            urllib.request.urlopen(url) as response,
            open(target_path, "wb") as out_file,
        ):
            out_file.write(response.read())
        print(f"Downloaded {filename} to {target_path}")
    except Exception as e:
        print(f"Failed to download {filename}. Error: {e}")
    return target_path


# Define the directories to search and the files to download
search_directories = [".", "../02_bonus_bytepair-encoder/gpt2_model/"]

files_to_download = {
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe": "vocab.bpe",
    "https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json": "encoder.json",
}

# Ensure directories exist and download files if needed
paths = {}
for url, filename in files_to_download.items():
    paths[filename] = download_file_if_absent(url, filename, search_directories)

Downloaded vocab.bpe to ./vocab.bpe
Downloaded encoder.json to ./encoder.json


Now we can load the files, check the vocabulary size and encode / decode a sample text.

In [29]:
tokenizer_gpt2 = BytePairEncoderSimple()
tokenizer_gpt2.load_vocab_and_merges_from_openai(
    vocab_path=paths["encoder.json"], bpe_merges_path=paths["vocab.bpe"]
)

print("Size of vocabulary:", len(tokenizer_gpt2.vocab))

# Encode and decode a sample text
input_text = "This is some text"
token_ids = tokenizer_gpt2.encode(input_text)
print("Token IDs:", token_ids)
print("Decoded text:", tokenizer_gpt2.decode(token_ids))

Size of vocabulary: 50257
Token IDs: [1212, 318, 617, 2420]
Decoded text: This is some text
